In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ezc3d import c3d
from scipy.signal import resample
from scipy.io import loadmat, savemat
from joblib import load, dump
from tqdm import tqdm

In [2]:
PATH_DATASETS = '../../Dataset/'
PATH_DATASET_1 = 'gait-dbase-1/'

N_SAMPLES = 1024
FORCE_THRESHOLD = 20
TOE_HEIGTH_THRESHOLD = 40

In [ ]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_1)
subjects = os.listdir(path)

features = pd.DataFrame()
target = pd.DataFrame()

records = []

for subject in tqdm(subjects):
    files_path = os.path.join(path, subject)
    files = os.listdir(files_path)
    for filename in files:

        try:
            content = c3d(os.path.join(path, subject, filename), extract_forceplat_data=True)
            trajectory_x_l = content['data']['points'][1, 9:15, :]
            trajectory_y_l = content['data']['points'][1, 9:15, :]
            trajectory_z_l = content['data']['points'][2, 9:15, :]

            trajectory_x_r = content['data']['points'][1, 20:26, :]
            trajectory_y_r = content['data']['points'][1, 20:26, :]
            trajectory_z_r = content['data']['points'][2, 20:26, :]

            trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=1).T
            trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=1).T
            trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=1).T

            trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=1).T
            trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=1).T
            trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=1).T

            force_x_1 = content['data']['platform'][0]['force'][0, :]
            force_y_1 = content['data']['platform'][0]['force'][1, :]
            force_z_1 = content['data']['platform'][0]['force'][2, :]

            force_x_2 = content['data']['platform'][1]['force'][0, :]
            force_y_2 = content['data']['platform'][1]['force'][1, :]
            force_z_2 = content['data']['platform'][1]['force'][2, :]

            force_x_1 = resample(force_x_1, N_SAMPLES, axis=0).T
            force_y_1 = resample(force_y_1, N_SAMPLES, axis=0).T
            force_z_1 = resample(force_z_1, N_SAMPLES, axis=0).T

            force_x_2 = resample(force_x_2, N_SAMPLES, axis=0).T
            force_y_2 = resample(force_y_2, N_SAMPLES, axis=0).T
            force_z_2 = resample(force_z_2, N_SAMPLES, axis=0).T

            force_combined_z = force_z_1 + force_z_2
            force_mask = (force_combined_z >= FORCE_THRESHOLD)

            trajectory_x_l = trajectory_x_l[force_mask, :]
            trajectory_y_l = trajectory_y_l[force_mask, :]
            trajectory_z_l = trajectory_z_l[force_mask, :]

            trajectory_x_r = trajectory_x_r[force_mask, :]
            trajectory_y_r = trajectory_y_r[force_mask, :]
            trajectory_z_r = trajectory_z_r[force_mask, :]

            trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=0)
            trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=0)
            trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=0)

            trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=0)
            trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=0)
            trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=0)

            force_x_1 = force_x_1[force_mask]
            force_y_1 = force_y_1[force_mask]
            force_z_1 = force_z_1[force_mask]

            force_x_2 = force_x_2[force_mask]
            force_y_2 = force_y_2[force_mask]
            force_z_2 = force_z_2[force_mask]

            force_x_1 = resample(force_x_1, N_SAMPLES, axis=0)
            force_y_1 = resample(force_y_1, N_SAMPLES, axis=0)
            force_z_1 = resample(force_z_1, N_SAMPLES, axis=0)

            force_x_2 = resample(force_x_2, N_SAMPLES, axis=0)
            force_y_2 = resample(force_y_2, N_SAMPLES, axis=0)
            force_z_2 = resample(force_z_2, N_SAMPLES, axis=0)

            force_x_l = 0
            force_y_l = 0
            force_z_l = 0
            force_x_r = 0
            force_y_r = 0
            force_z_r = 0

            fm1_z_l = trajectory_z_l[:, 3]
            indices = np.arange(0, len(fm1_z_l))
            peak_height_idx = np.argmax(fm1_z_l)
            force_mask_1 = (force_z_1 >= FORCE_THRESHOLD)
            area_of_interest = indices[force_mask_1]

            if peak_height_idx in area_of_interest:
                force_x_l = force_x_2
                force_x_r = force_x_1
                force_y_l = force_y_2
                force_y_r = force_y_1
                force_z_l = force_z_2
                force_z_r = force_z_1

            else:
                force_x_l = force_x_1
                force_x_r = force_x_2
                force_y_l = force_y_1
                force_y_r = force_y_2
                force_z_l = force_z_1
                force_z_r = force_z_2
            
            
            _features_l = pd.DataFrame(
                np.concatenate([trajectory_x_l, trajectory_y_l, trajectory_z_l], axis=1),
                columns=[
                    'fal_x', 'tam_x', 'fcc_x', 'fm1_x', 'fm2_x', 'fm5_x',
                    'fal_y', 'tam_y', 'fcc_y', 'fm1_y', 'fm2_y', 'fm5_y',
                    'fal_z', 'tam_z', 'fcc_z', 'fm1_z', 'fm2_z', 'fm5_z'
                ]
            )

            _features_r = pd.DataFrame(
                np.concatenate([trajectory_x_r, trajectory_y_r, trajectory_z_r], axis=1),
                columns=[
                    'fal_x', 'tam_x', 'fcc_x', 'fm1_x', 'fm2_x', 'fm5_x',
                    'fal_y', 'tam_y', 'fcc_y', 'fm1_y', 'fm2_y', 'fm5_y',
                    'fal_z', 'tam_z', 'fcc_z', 'fm1_z', 'fm2_z', 'fm5_z'
                ]
            )

            _features = pd.concat([_features_l, _features_r], axis=0)

            _target_r = pd.DataFrame(
                np.stack([force_x_r, force_y_r, force_z_r], axis=1),
                columns=['fx', 'fy', 'fz']
            )

            _target_l = pd.DataFrame(
                np.stack([force_x_l, force_y_l, force_z_l], axis=1),
                columns=['fx', 'fy', 'fz']
            )

            _target = pd.concat([_target_r, _target_l], axis=0)

            features = pd.concat([features, _features], axis=0, ignore_index=True)
            target = pd.concat([target, _target], axis=0, ignore_index=True)

            records.append(filename)

        except Exception as e:
            # print('Error occured', e)
            pass



In [ ]:
features

In [ ]:
target

In [ ]:
path = '../../Dataset/Processed/data.joblib'
data = {
    'X': features,
    'y': target
}
dump(data, path)

In [ ]:
features_z = features[['fal_z', 'tam_z', 'fcc_z', 'fm1_z', 'fm2_z', 'fm5_z']].to_numpy()
target_z = target[['fz']].to_numpy()

In [ ]:
X = features_z.reshape(-1, 1024, 6)
y = target_z.reshape(-1, 1024, 1)

In [ ]:
sample = 1116

plt.figure(figsize=(10, 3))
# plt.plot(X[sample, :, 1])
y1 = y[1110, :, 0].T
y2 = y[1111, :, 0].T
plt.plot(y1 + y2)
# plt.plot(y2)
plt.title('VGRF')
plt.xlabel('Time')
plt.legend(['SUM'])
plt.show()

In [ ]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_1)
files_path = os.path.join(path, subjects[0])
file_path = os.path.join(files_path, records[1])

In [ ]:
content = c3d(file_path, extract_forceplat_data=True)

trajectory_x_l = content['data']['points'][1, 9:15, :]
trajectory_y_l = content['data']['points'][1, 9:15, :]
trajectory_z_l = content['data']['points'][2, 9:15, :]

trajectory_x_r = content['data']['points'][1, 20:26, :]
trajectory_y_r = content['data']['points'][1, 20:26, :]
trajectory_z_r = content['data']['points'][2, 20:26, :]

trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=1).T
trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=1).T
trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=1).T

trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=1).T
trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=1).T
trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=1).T

force_x_1 = content['data']['platform'][0]['force'][0, :]
force_y_1 = content['data']['platform'][0]['force'][1, :]
force_z_1 = content['data']['platform'][0]['force'][2, :]

force_x_2 = content['data']['platform'][1]['force'][0, :]
force_y_2 = content['data']['platform'][1]['force'][1, :]
force_z_2 = content['data']['platform'][1]['force'][2, :]

force_x_1 = resample(force_x_1, N_SAMPLES, axis=0).T
force_y_1 = resample(force_y_1, N_SAMPLES, axis=0).T
force_z_1 = resample(force_z_1, N_SAMPLES, axis=0).T

force_x_2 = resample(force_x_2, N_SAMPLES, axis=0).T
force_y_2 = resample(force_y_2, N_SAMPLES, axis=0).T
force_z_2 = resample(force_z_2, N_SAMPLES, axis=0).T

In [ ]:
plt.plot((force_z_2))